In [1]:
from copy import deepcopy
import os

from pprint import PrettyPrinter
import xml.etree.ElementTree as ET
import xmltodict

from resources.conversion import (
    convert_ammo_config_py2xml, convert_weapon_config_py2xml, convert_pickup_item,
    AMMOS_CONVERSION, WEAPONS_CONVERSION,
)

PATH_INPUTS = './weapon-merger/composed/merged'
PATH_OUTPUTS = './outputs'
DELETE_TOKEN = '!DELETE'

pp = PrettyPrinter(indent=1, compact=True, sort_dicts=False)
def pprint(obj):
    return pp.pprint(obj)

In [ ]:
# Convert weapons.meta with Pickups mod conversion policy
weapons_merged_meta = xmltodict.parse(
    xml_input=ET.tostring(
        element=ET.parse(
            source=f'{PATH_INPUTS}/update/update.rpf/common/data/ai/weapons.meta'
        ).getroot()
    )
)

weapons_converted_meta = deepcopy(weapons_merged_meta)
DELETE_TOKEN = '!DELETE'
for i_a, ammo in enumerate(list(weapons_merged_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'])):
    for k_convert in list(AMMOS_CONVERSION[ammo['Name']].keys()):
        assert k_convert in ['add', 'update', 'delete', 'metadata'], f'Unknown conversion type {k_convert} for weapons.'
        # `add`` to create clones and fork to multiple versions
        if k_convert == 'add':
            for u in AMMOS_CONVERSION[ammo['Name']]['add']:
                template:dict = deepcopy(ammo)
                template.update(convert_ammo_config_py2xml(u))
                weapons_converted_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'].append(template)
        # `update`` to update in-place
        elif k_convert == 'update':
            template:dict = deepcopy(ammo)
            template.update(convert_ammo_config_py2xml(AMMOS_CONVERSION[ammo['Name']]['update']))
            weapons_converted_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'][i_a] = template
        # `delete`` to delete the item
        elif k_convert == 'delete':
            weapons_converted_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'][i_a] = DELETE_TOKEN

for i_w, weapon in enumerate(list(weapons_merged_meta['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'])):
    for k_convert in list(WEAPONS_CONVERSION[weapon['Name']].keys()):
        assert k_convert in ['update', 'metadata'], f'Unknown conversion type {k_convert} for weapons.'
        if k_convert == 'update':
            template:dict = deepcopy(weapon)
            template.update(convert_weapon_config_py2xml(WEAPONS_CONVERSION[weapon['Name']]['update']))
            weapons_converted_meta['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'][i_w] = template

while DELETE_TOKEN in weapons_converted_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item']:
    weapons_converted_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'].remove(DELETE_TOKEN)
while DELETE_TOKEN in weapons_converted_meta['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item']:
    weapons_converted_meta['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'].remove(DELETE_TOKEN)

# Export converted weapons info to .meta
if True:
    f = f'{PATH_OUTPUTS}/update/update.rpf/common/data/ai/weapons.meta'
    if not os.path.isdir(os.path.dirname(f)):
        os.makedirs(os.path.dirname(f))

    weapons_converted_meta_xml = ET.ElementTree(element=ET.fromstring(text=xmltodict.unparse(
        input_dict=weapons_converted_meta,
        encoding='utf-8',
        pretty=True,
        indent=2,
    )))
    weapons_converted_meta_xml.write(
        file_or_filename=f,
        encoding="utf-8",
        xml_declaration=True,
    )
    print(f)

In [ ]:
# Convert weapons.meta with Pickups mod conversion policy
pickups_merged_meta = xmltodict.parse(
    xml_input=ET.tostring(
        element=ET.parse(
            source=f'{PATH_INPUTS}/update/update.rpf/common/data/pickups.meta'
        ).getroot()
    )
)


pickups_converted_meta = deepcopy(pickups_merged_meta)
for i_p, pickup in enumerate(list(pickups_converted_meta['CPickupDataManager']['pickupData']['Item'])):
    pickups_converted_meta['CPickupDataManager']['pickupData']['Item'][i_p] = convert_pickup_item(pickup)

if True:
    f = f'{PATH_OUTPUTS}/update/update.rpf/common/data/pickups.meta'
    pickups_converted_meta_xml = ET.ElementTree(element=ET.fromstring(text=xmltodict.unparse(
        input_dict=pickups_converted_meta,
        encoding='utf-8',
        pretty=True,
        indent=2,
    )))
    pickups_converted_meta_xml.write(
        file_or_filename=f,
        encoding="utf-8",
        xml_declaration=True,
    )
    print(f)